In [1]:
import pandas 

kinetics = pandas.read_csv( 'kinetics.csv' )
thermal = pandas.read_csv( 'thermal.csv' ) 

print( kinetics.columns )
print( thermal.columns ) 

Index(['well', 'rate', 'mutant', 'protein_yield', 'dilution', 'note'], dtype='object')
Index(['well', 'rate', 'mutant'], dtype='object')


In [2]:
# for thermal 
# convert well indexes into temperatures based on map 

temp_key = pandas.read_csv( '../raw/temperature_key.csv', index_col='Well' )
thermal['temp'] = thermal.well.str[0].map( temp_key.Celsius.to_dict() ) 

print( thermal.sample( 3 ) )

     well     rate mutant   temp
707    H5  0.33700  K341A  30.05
1411   C1  0.00046  S331A  44.30
1792   C7  0.00260  W409Y  44.30


In [3]:
# for kinetics, convert the rate into units (1/min) 
# hard-coded values:
#  0.0002 is the conversion from OD to M
#  0.25 is because we dilute the protein 4-fold upon initiation 
#  113330 is the extinction coef of BglB, used to convert mg/mL to M

kinetics[ 'kobs' ] = kinetics['rate'] * 0.0002 / ( kinetics[ 'protein_yield' ] * kinetics[ 'dilution' ] * 0.25 / 113300 )

print( kinetics.sample( 3 ) ) 

    well      rate mutant  protein_yield  dilution  note       kobs
126   G7  0.006700  N220Y           1.84      0.01   NaN  33.004783
31    H4 -0.000009  N293C           1.87      0.10   NaN  -0.004265
179   D2  0.001630  N404C           1.64      0.01   NaN   9.008732


In [4]:
# also for kinetics, convert well indexes into substrate concentrations

def my_s( x ):
    my_ss = dict( zip( 'ABCDEFGH', [ .075, .01875, .0046875, .001171875, .0002929688, .0000732422, .0000183105, 0 ] ) )
    return my_ss[ x[0] ] 

kinetics[ 's' ] = kinetics.well.map( my_s ) 

In [5]:
import numpy as np 
np.mean( [ .75, .1875, .046875, .01171875, .002929688, .000732422, .000183105, 0 ] )

0.124992370625

In [6]:
# anything we need to manually drop? 

to_drop = [ 'WT', 'WT1' ] 

for df in [ kinetics, thermal ]:
    for item in to_drop:
        if item in df.index:
            df.drop( item, inplace=True ) 

In [7]:
# now export "cleaned" data 

kinetics.to_csv( '../pub/kinetics.csv' ) 
thermal.to_csv( '../pub/thermal.csv' ) 